# 4. Гибридные модели

In [1]:
# conda install lightfm
# !pip install lightfm
# %conda install -c conda-forge lightfm

In [22]:
import pandas as pd
import numpy as np

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k   

In [3]:
ratings = pd.read_csv('data/Gooddreadbooks/ratings.csv') # Поставленные оценки
books = pd.read_csv('data/Gooddreadbooks/books.csv') # Информация о книгах
tags = pd.read_csv('data/Gooddreadbooks/tags.csv') # Информация о тегах
book_tags = pd.read_csv('data/Gooddreadbooks/book_tags.csv') # Книги с тегами 

In [4]:
books.head(3)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...


In [5]:
book_tags.head(3)

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173


### Задание 4.1

Добавьте в набор данных book_tags признак с обычным id книги, используя соответствие обычного id и id в системе Goodreads.

Какой обычный id у книги, которая имеет id 5 в системе Goodreads?

In [6]:
# book_tags = book_tags.merge(books[['book_id', 'goodreads_book_id']], on='goodreads_book_id', how='left', indicator=True)
book_tags = book_tags.merge(books[['book_id', 'goodreads_book_id']], on='goodreads_book_id', how='left', indicator=False)
book_tags.head(3)

,goodreads_book_id,tag_id,count,book_id
0,1,30574,167697,27
1,1,11305,37174,27
2,1,11557,34173,27


In [7]:
dict(book_tags[(book_tags['goodreads_book_id'] == 5)].iloc[0])

{'goodreads_book_id': 5, 'tag_id': 11557, 'count': 40087, 'book_id': 18}

### Задание 4.2

Далее нам необходимо оставить в наборе данных book_tags только те записи, теги для которых есть в данных tags.

Отфильтруйте данные таким образом, чтобы в наборе данных book_tags остались только те строки, в которых находятся теги, информация о которых есть в наборе данных tags.

Сколько объектов осталось?

In [8]:
tags.head(3)

,tag_id,tag_name
0,509,19th-century
1,923,20th-century
2,941,21st-century


In [9]:
book_tags = book_tags.merge(tags, on='tag_id', how='left', indicator=False)
book_tags.head(3)

,goodreads_book_id,tag_id,count,book_id,tag_name
0,1,30574,167697,27,NaN
1,1,11305,37174,27,fantasy
2,1,11557,34173,27,NaN


In [10]:
print(len(book_tags['tag_name']), 'До фильтра')

book_tags.dropna(subset=['tag_name'], inplace=True)

print(len(book_tags['tag_name']), 'После фильтра')

999912 До фильтра
300738 После фильтра


In [11]:
from scipy.sparse import csr_matrix

Нам важно преобразовать данные в специальный формат, в котором хранятся разрежённые матрицы — будем использовать формат Compressed Sparse Row (CSR), подразумевающий подсчёт кумулятивной суммы количества элементов в строке вместо индексов строк.

Осуществляем преобразование следующим образом:

In [12]:
# Передаём в качестве аргументов в функцию выставленный рейтинг (это будут значения матрицы), 
# а также id пользователя и id книги (это будут индексы для строк и столбцов матрицы)

ratings_matrix = csr_matrix((ratings.rating,(ratings.user_id,ratings.book_id))) 

Теперь нам необходимо составить матрицу с метаданными. В качестве индексов будут выступать id книги и id тега, и если у этой книги есть рассматриваемый тег, то на пересечении соответствующих строки и столбца будет выставлена единица.

In [13]:
meta_matrix  = csr_matrix(([1]*len(book_tags),(book_tags.book_id, book_tags.tag_id))) 

In [14]:
ratings_matrix.mean()

0.007086188900997592

Отлично, данные подготовлены — теперь настало время определить модель, которую мы будем использовать. Сделаем это следующим образом:

In [15]:
model = LightFM(
    loss='warp-kos', # Определяем функцию потерь
    random_state=42, # Фиксируем случайное разбиение
    learning_rate=0.05, # Темп обучения
    no_components=100 # Размерность вектора для представления данных в модели
)

В качестве функции потерь мы выбрали значение 'warp', хотя, разумеется, это не единственный вариант. В модуле LightFM представлены следующие функции потерь:

* 'logistic' — логистическая функция. Полезна в случаях, когда есть как положительные, так и отрицательные взаимодействия, например 1 и -1.
* 'bpr' — байесовский персонализированный рейтинг. Можно применять, когда присутствуют только положительные взаимодействия.
* 'warp' — парный взвешенный приблизительный ранг. Используется, если необходимо повысить качество именно в верхней части списка рекомендаций.
* 'warp-kos' — модификация warp.

In [16]:
train, test = random_train_test_split(
    ratings_matrix, # Общая выборка
    test_percentage=0.2, # Размер тестовой выборки
    random_state=42 # Генератор случайных чисел
)

Теперь обучим модель на наших данных о взаимодействии, также используя метаданные о книгах. Для этого воспользуемся методом fit(). В этот метод передадим обучающую выборку, признаки товаров — item_features, количество эпох обучения (сколько раз мы будем показывать модели исходный датасет, чтобы она лучше выучила данные) — epochs, а также параметр verbose для отслеживания процесса обучения:

In [17]:
model = model.fit(
    train, # Обучающая выборка
    item_features=meta_matrix, # Признаки товаров
    epochs=10, # Количество эпох
    verbose=True # Отображение обучения
)

Epoch: 100%|██████████| 10/10 [22:46<00:00, 136.62s/it]


### Задание 4.5

Оцените качество полученной модели с помощью функции precision_at_k, передав в неё три аргумента: модель, тестовые данные и обозначение метаданных (item_features = meta_matrix).

Примечание. Процесс расчёта метрик рекомендательной системы также является довольно затратным по времени. Для ускорения этого процесса вы можете передать параметр num_threads, чтобы указать количество потоков процессора, используемых для вычислений.

Выведите среднее арифметическое и округлите его до двух знаков после точки-разделителя.

In [20]:
prec_score = precision_at_k(
                     model,
                     test,
                     num_threads=20,
                     item_features = meta_matrix).mean() 
print(prec_score)

0.023422552


In [25]:
scores = model.predict(30574, np.arange(27), user_features=meta_matrix)

scores

Exception: Number of user feature rows does not equal the number of users